<img align="right" src="../../additional_data/banner_siegel.png" style="width:1000px;">

# Xarray-I: Data Structure 

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Prerequisites**: There is no prerequisite learning required.


## Background

In the previous notebook, we experienced that the data we wanna access are loaded in a form called **`xarray.dataset`**. This is the form in which earth observation data are usually stored in a datacube.

**`xarray`** is an open source project and Python package which offers a toolkit for working with ***multi-dimensional arrays*** of data. **`xarray.dataset`** is an in-memory representation of a netCDF (network Common Data Form) file. Understanding the structure of a **`xarray.dataset`** is the key to enable us work with these data. Thus, in this notebook, we are mainly dedicated to helping users of our datacube understand its data structure.

Firstly let's come to the end stage of the previous notebook, where we have loaded a data product. The data product "s2_l2a_bavaria" is used as example in this notebook.

## Description

The following topics are convered in this notebook:
* **What is inside a `xrray.dataset` (the structure)?**
* **(Basic) Subset Dataset / DataArray**
* **Reshape a Dataset**

In [1]:
import datacube
import pandas as pd
from odc.ui import DcViewer 
from odc.ui import with_ui_cbk
import xarray as xr
import matplotlib.pyplot as plt

# Set config for displaying tables nicely
# !! USEFUL !! otherwise parts of longer infos won't be displayed in tables
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

# Connect to DataCube
# argument "app" --- user defined name for a session (e.g. choose one matching the purpose of this notebook)
dc = datacube.Datacube(app = "nb_understand_ndArrays")

In [2]:
# Load Data Product
ds = dc.load(product= "s2_l2a",
            x= [12.94 ,13.05],
             y= [53.88,53.94],
             output_crs = "EPSG:32632",
             time = ("2020-10-01", "2020-12-31"),
             measurements= ["blue", "green", "red","nir"],
             resolution = (-10,10),
             group_by = "solar_day",
             progress_cbk=with_ui_cbk())

ds

/opt/conda/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class CommonTimestamp will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self._connection.execute(select_query)
/opt/conda/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class Float8Range will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  

<xarray.Dataset>
Dimensions:      (time: 34, y: 739, x: 1286)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-03T10:25:59 ... 2020-12-29T10:...
  * y            (y) float64 5.985e+06 5.985e+06 ... 5.977e+06 5.977e+06
  * x            (x) float64 7.586e+05 7.586e+05 ... 7.714e+05 7.714e+05
    spatial_ref  int32 32632
Data variables:
    blue         (time, y, x) uint16 1032 999 899 903 ... 2358 2366 2272 2164
    green        (time, y, x) uint16 1240 1184 957 931 ... 1930 1864 1848 1848
    red          (time, y, x) uint16 1186 1094 850 797 ... 1772 1806 1766 1718
    nir          (time, y, x) uint16 3068 2806 1788 1606 ... 2070 2066 1970 1982
Attributes:
    crs:           EPSG:32632
    grid_mapping:  spatial_ref

In [ ]:
#da = ds.to_array().rename({"variable":"band"})
#print(da)

In [ ]:
#ds2 = da.to_dataset(dim="time")
#ds2

## **What is inside a `xarray.dataset`?**
The figure below is a diagramm depicting the structure of the **`xarray.dataset`** we've just loaded. Combined with the diagramm, we hope you may better interpret the texts below explaining the data strucutre of a **`xarray.dataset`**.

![xarray data structure](https://live.staticflickr.com/65535/51083605166_70dd29baa8_k.jpg)

As read from the output block, this dataset has three ***Data Variables*** , "blue", "green" and "red" (shown with colors in the diagramm), referring to individual spectral band.

Each data variable can be regarded as a **multi-dimensional *Data Array*** of same structure; in this case, it is a **three-dimensional array** (shown as 3D Cube in the diagramm) where `time`, `x` and `y` are its ***Dimensions*** (shown as axis along each cube in the diagramm).

In this dataset, there are 35 ***Coordinates*** under `time` dimension, which means there are 35 time steps along the `time` axis. There are 164 coordinates under `x` dimension and 82 coordinates under `y` dimension, indicating that there are 164 pixels along `x` axis and 82 pixels along `y` axis.

As for the term ***Dataset***, it is like a *Container* holding all the multi-dimensional arrays of same structure (shown as the red-lined box holding all 3D Cubes in the diagramm).

So this instance dataset has a spatial extent of 164 by 82 pixels at given lon/lat locations, spans over 35 time stamps and 3 spectral band.

**In summary, *`xarray.dataset`* is substantially a container for high-dimensional *`DataArray`* with common attributes (e.g. crs) attached**, :
* **Data Variables (`values`)**: **it's generally the first/highest dimension to subset from a high dimensional array.** Each `data variable` contains a multi-dimensional array of all other dimensions.
* **Dimensions (`dims`)**: other dimensions arranged in hierachical order *(e.g. 'time', 'y', 'x')*.
* **Coordinates (`coords`)**: Coordinates along each `Dimension` *(e.g. timesteps along 'time' dimension, latitudes along 'y' dimension, longitudes along 'x' dimension)*
* **Attributes (`attrs`)**: A dictionary(`dict`) containing Metadata.

Now let's deconstruct the dataset we have just loaded a bit further to have things more clarified!:D

* **To check existing dimensions of a dataset**

In [7]:
ds.dims

Frozen({'time': 21, 'y': 1031, 'x': 1010})

* **To check the coordinates of a dataset**

In [8]:
ds.coords#['time']

Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734

* **To check all coordinates along a specific dimension**
<br>
<img src=https://live.staticflickr.com/65535/51115452191_ec160d4514_o.png, width="450">

In [9]:
ds.time
# OR
#ds.coords['time']

<xarray.DataArray 'time' (time: 21)>
array(['2020-10-01T08:28:17.000000000', '2020-10-11T08:28:18.000000000',
       '2020-10-16T08:28:20.000000000', '2020-10-21T08:28:18.000000000',
       '2020-10-26T08:28:19.000000000', '2020-10-31T08:28:17.000000000',
       '2020-11-05T08:28:19.000000000', '2020-11-10T08:28:15.000000000',
       '2020-11-13T08:38:10.000000000', '2020-11-15T08:28:17.000000000',
       '2020-11-20T08:28:15.000000000', '2020-11-25T08:28:15.000000000',
       '2020-11-30T08:28:14.000000000', '2020-12-03T08:38:08.000000000',
       '2020-12-05T08:28:12.000000000', '2020-12-08T08:38:07.000000000',
       '2020-12-10T08:28:11.000000000', '2020-12-13T08:38:06.000000000',
       '2020-12-15T08:28:12.000000000', '2020-12-20T08:28:10.000000000',
       '2020-12-30T08:28:12.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

* **To check attributes of the dataset**

In [10]:
ds.attrs

{'crs': 'EPSG:32734', 'grid_mapping': 'spatial_ref'}

## **Subset Dataset / DataArray**

* **To select all data of "blue" band**
<br>
<img src=https://live.staticflickr.com/65535/51115092614_366cb774a8_o.png, width="350">

In [13]:
ds.blue
# OR
#ds['blue']

<xarray.DataArray 'blue' (time: 21, y: 1031, x: 1010)>
array([[[  800,   955,  1188, ...,  8856,  9152,  9184],
        [  860,  1036,  1188, ...,  9040,  9360,  9304],
        [  920,  1214,  1222, ...,  9200,  9432,  9240],
        ...,
        [  933,   966,  1017, ...,   755,   881,  1009],
        [ 1040,  1019,  1009, ...,   837,   864,   969],
        [ 1102,  1088,  1034, ...,   697,   807,   954]],

       [[ 1018,  1032,  1032, ...,   752,   738,   771],
        [ 1036,  1058,  1023, ...,   763,   747,   751],
        [ 1036,  1062,  1026, ...,   686,   693,   707],
        ...,
        [  731,   735,   788, ...,   715,   641,   750],
        [  830,   868,   846, ...,   651,   678,   686],
        [  921,   918,   895, ...,   484,   496,   693]],

       [[ 1052,  1078,  1050, ...,   782,   811,   795],
        [ 1080,  1096,  1040, ...,   760,   791,   764],
        [ 1070,  1052,  1032, ...,   712,   754,   749],
        ...,
...
        ...,
        [  695,   648,   586, ...,   454,   583,   690],
        [  644,   641,   567, ...,   568,   558,   648],
        [  589,   601,   544, ...,   452,   507,   671]],

       [[  802,   810,   821, ...,   742,   763,   758],
        [  809,   831,   825, ...,   753,   769,   762],
        [  851,   850,   828, ...,   727,   756,   783],
        ...,
        [  808,   744,   731, ...,   714,   800,   841],
        [  797,   735,   729, ...,   648,   717,   830],
        [  759,   761,   716, ...,   695,   709,   803]],

       [[11848, 11992, 11800, ...,  7364,  7276,  7212],
        [11808, 11760, 11672, ...,  7192,  7100,  7076],
        [11688, 11592, 11504, ...,  7124,  7008,  6952],
        ...,
        [ 8248,  8344,  8336, ...,  2474,  2512,  2586],
        [ 8288,  8312,  8288, ...,  2472,  2500,  2630],
        [ 8288,  8312,  8336, ...,  2502,  2552,  2722]]], dtype=uint16)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

In [14]:
# Only print pixel values
ds.blue.values

array([[[  800,   955,  1188, ...,  8856,  9152,  9184],
        [  860,  1036,  1188, ...,  9040,  9360,  9304],
        [  920,  1214,  1222, ...,  9200,  9432,  9240],
        ...,
        [  933,   966,  1017, ...,   755,   881,  1009],
        [ 1040,  1019,  1009, ...,   837,   864,   969],
        [ 1102,  1088,  1034, ...,   697,   807,   954]],

       [[ 1018,  1032,  1032, ...,   752,   738,   771],
        [ 1036,  1058,  1023, ...,   763,   747,   751],
        [ 1036,  1062,  1026, ...,   686,   693,   707],
        ...,
        [  731,   735,   788, ...,   715,   641,   750],
        [  830,   868,   846, ...,   651,   678,   686],
        [  921,   918,   895, ...,   484,   496,   693]],

       [[ 1052,  1078,  1050, ...,   782,   811,   795],
        [ 1080,  1096,  1040, ...,   760,   791,   764],
        [ 1070,  1052,  1032, ...,   712,   754,   749],
        ...,
        [  789,   808,   839, ...,   624,   626,   808],
        [  908,   909,   875, ...,   671,   6

* **To select blue band data at the first time stamp**
<br>
<img src=https://live.staticflickr.com/65535/51116131265_8464728bc1_o.png, width="350">

In [15]:
ds.blue[0]

<xarray.DataArray 'blue' (y: 1031, x: 1010)>
array([[ 800,  955, 1188, ..., 8856, 9152, 9184],
       [ 860, 1036, 1188, ..., 9040, 9360, 9304],
       [ 920, 1214, 1222, ..., 9200, 9432, 9240],
       ...,
       [ 933,  966, 1017, ...,  755,  881, 1009],
       [1040, 1019, 1009, ...,  837,  864,  969],
       [1102, 1088, 1034, ...,  697,  807,  954]], dtype=uint16)
Coordinates:
    time         datetime64[ns] 2020-10-01T08:28:17
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select blue band data at the first time stamp while the latitude is the largest in the defined spatial extent**
<img src=https://live.staticflickr.com/65535/51115337046_aeb75d0d03_o.png, width="350">

In [16]:
ds.blue[0][0]

<xarray.DataArray 'blue' (x: 1010)>
array([ 800,  955, 1188, ..., 8856, 9152, 9184], dtype=uint16)
Coordinates:
    time         datetime64[ns] 2020-10-01T08:28:17
    y            float64 6.807e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select the upper-left corner pixel**
<br>
<img src=https://live.staticflickr.com/65535/51116131235_b0cca9589f_o.png, width="350">

In [17]:
ds.blue[0][0][0]

<xarray.DataArray 'blue' ()>
array(800, dtype=uint16)
Coordinates:
    time         datetime64[ns] 2020-10-01T08:28:17
    y            float64 6.807e+06
    x            float64 8.687e+05
    spatial_ref  int32 32734
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

### **subset dataset with `isel` vs. `sel`**
* Use `isel` when subsetting with **index**
* Use `sel` when subsetting with **labels**

* **To select data of all spectral bands at the first time stamp**
<br>
<img src=https://live.staticflickr.com/65535/51114879732_7d62db54f4_o.png, width="750">

In [18]:
ds.isel(time=[0])

<xarray.Dataset>
Dimensions:      (time: 1, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 800 955 1188 1404 1584 ... 867 697 807 954
    green        (time, y, x) uint16 856 1044 1222 1444 ... 1114 932 1054 1246
    red          (time, y, x) uint16 927 1154 1326 1552 ... 1556 1174 1418 1840
    nir          (time, y, x) uint16 1128 1324 1572 1824 ... 2204 2222 2292 2388
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select data of all spectral bands of year 2020** 
<br>
<img src=https://live.staticflickr.com/65535/51116281070_75f1b46a9c_o.png, width="750">

In [19]:
ds.sel(time='2020-12')
#print(ds.sel(time='2019'))

<xarray.Dataset>
Dimensions:      (time: 8, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-12-03T08:38:08 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 724 713 706 699 704 ... 2574 2502 2552 2722
    green        (time, y, x) uint16 1004 988 985 996 ... 2378 2508 2426 2432
    red          (time, y, x) uint16 1192 1184 1200 1184 ... 2402 2332 2290 2348
    nir          (time, y, x) uint16 2454 2464 2332 2316 ... 3286 3348 3378 3398
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

***Tip: More about indexing and sebsetting Dataset or DataArray is presented in the [Notebook_05](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/05_xarrayII.ipynb).***

## **Reshape Dataset**

* **Convert the Dataset (subset to 2019) to a *4-dimension* DataArray**

In [20]:
da = ds.sel(time='2020-12').to_array().rename({"variable":"band"})
da

<xarray.DataArray (band: 4, time: 8, y: 1031, x: 1010)>
array([[[[  724,   713,   706, ...,     0,     0,     0],
         [  682,   709,   729, ...,     0,     0,     0],
         [  749,   745,   738, ...,     0,     0,     0],
         ...,
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],

        [[  797,   797,   792, ...,   714,   736,   727],
         [  812,   807,   815, ...,   726,   730,   717],
         [  833,   842,   806, ...,   710,   710,   730],
         ...,
         [  912,   833,   815, ...,   714,   666,   812],
         [  877,   807,   811, ...,   738,   719,   766],
         [  742,   761,   760, ...,   609,   645,   774]],

        [[14296, 14376, 14352, ...,     0,     0,     0],
         [14376, 14344, 14376, ...,     0,     0,     0],
         [14376, 14424, 14416, ...,     0,     0,     0],
         ...,
...
         ...,
         [ 2880,  2814,  2846, ...,  2406,  2358,  2288],
         [ 2940,  2790,  2674, ...,  2266,  2216,  2354],
         [ 3162,  3050,  2872, ...,  2254,  2388,  2414]],

        [[ 2388,  2326,  2302, ...,  2850,  2828,  2798],
         [ 2332,  2338,  2366, ...,  2746,  2746,  2792],
         [ 2428,  2412,  2442, ...,  2708,  2736,  2734],
         ...,
         [ 2800,  2762,  2824, ...,  2532,  2420,  2416],
         [ 2834,  2692,  2630, ...,  2470,  2406,  2498],
         [ 3010,  2850,  2700, ...,  2380,  2462,  2496]],

        [[10096, 10104, 10008, ...,  6788,  6768,  6712],
         [ 9984,  9992,  9880, ...,  6764,  6680,  6624],
         [ 9912,  9880,  9864, ...,  6704,  6624,  6608],
         ...,
         [ 7548,  7520,  7544, ...,  3360,  3344,  3372],
         [ 7520,  7580,  7548, ...,  3372,  3398,  3462],
         [ 7520,  7520,  7560, ...,  3348,  3378,  3398]]]], dtype=uint16)
Coordinates:
  * time         (time) datetime64[ns] 2020-12-03T08:38:08 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
  * band         (band) <U5 'blue' 'green' 'red' 'nir'
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **Convert the *4-dimension* DataArray back to a Dataset by setting the "time" as DataVariable (reshaped)**

![ds_reshaped](https://live.staticflickr.com/65535/51151694092_ca550152d6_o.png)

In [21]:
ds_reshp = da.to_dataset(dim="time")
print(ds_reshp)

<xarray.Dataset>
Dimensions:              (band: 4, y: 1031, x: 1010)
Coordinates:
  * y                    (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x                    (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref          int32 32734
  * band                 (band) <U5 'blue' 'green' 'red' 'nir'
Data variables:
    2020-12-03 08:38:08  (band, y, x) uint16 724 713 706 699 704 ... 0 0 0 0 0
    2020-12-05 08:28:12  (band, y, x) uint16 797 797 792 796 ... 2486 2648 2690
    2020-12-08 08:38:07  (band, y, x) uint16 14296 14376 14352 14376 ... 0 0 0 0
    2020-12-10 08:28:11  (band, y, x) uint16 633 641 639 648 ... 4704 4688 4780
    2020-12-13 08:38:06  (band, y, x) uint16 571 556 575 561 552 ... 0 0 0 0 0
    2020-12-15 08:28:12  (band, y, x) uint16 646 648 656 654 ... 2254 2388 2414
    2020-12-20 08:28:10  (band, y, x) uint16 802 810 821 810 ... 2380 2462 2496
    2020-12-30 08:28:12  (band, y, x) uint16 11848 11992 11800 ... 3378 3398
Att

## Recommended next steps

If you now understand the **data structure** of `xarray.dataset` and **basic indexing** methods illustrated in this notebook, you are ready to move on to the next notebook where you will learn more about **advanced indexing** and calculating some **basic statistical parameters** of the n-dimensional arrays!:D

In case you are gaining interest in exploring the world of **xarrays**, you may lay yourself into the [Xarray user guide](http://xarray.pydata.org/en/stable/index.html).

<br>
To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/01_jupyter_introduction.ipynb)
2. [eo2cube](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/02_eo2cube_introduction.ipynb)
3. [Loading Data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/03_data_lookup_and_loading.ipynb)
4. ***Xarray I: Data Structure (this notebook)***
5. [Xarray II: Index and Statistics](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/05_xarrayII.ipynb)
6. [Plotting data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/06_plotting_basics.ipynb)
7. [Spatial analysis](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/07_basic_analysis.ipynb)
8. [Parallel processing with Dask](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/08_parallel_processing_with_dask.ipynb)

The additional notebooks are designed for users to build up both basic and advanced skills which are not covered by the beginner's guide. Self-motivated users can go through them according to their own needs. They act as complements for the guide:
<br>

1. [Python's file management tools](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/I_file_management.ipynb)
2. [Image Processing basics using NumPy and Matplotlib](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/II_numpy_image_processing.ipynb)
3. [Vector Processing](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/III_process_vector_data.ipynb)
4. [Advanced Plotting](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/IV_advanced_plotting.ipynb)

***
## Additional information

This notebook is for the usage of Jupyter Notebook of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/).

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** April 2021